In [44]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
#import sys
from pyuvdata import UVData #, UVCal
#import hera_cal as hc
#from pyuvdata import utils as uvutils
import copy
#import uvtools as uvt
#from hera_cal.datacontainer import DataContainer
#import aipy
#import operator
from astropy import units as u
#from astropy import constants as c
import pandas as pd
from astropy.coordinates import Angle

In [65]:
JD = '2457549'
#rawpath = '/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/'+JD+'/'
path = '/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/'+JD+'/'

In [66]:
summary = pd.read_csv(os.path.join(path,JD+'_summary.csv'))

In [67]:
filenames = np.array(summary['Filename'])
lsts = np.array(summary['LST (rad)'])
jds = np.array(summary['Julian Date'])

In [68]:
# Set up LST grid
lst_bin_size = 6.*u.min
n_lst_bins = int((24.*u.hr/lst_bin_size).to(u.dimensionless_unscaled).value)
lst_edges = np.linspace(0, 2.*np.pi, n_lst_bins+1, endpoint=True)
lst_start = lst_edges[0:-1]
lst_end = lst_edges[1:]
lst_mid = (lst_start + lst_end)/2.

assert len(lst_start) == n_lst_bins
assert len(lst_end) == n_lst_bins
assert np.isclose((lst_end-lst_start)*24.*60./(2.*np.pi), lst_bin_size.value).sum() == n_lst_bins

In [69]:
one_second = 1/(24.*3600.)

In [70]:
%%time
# Probably want to calculate how many time samples *should* appear ...
min_time_samp = 1
for i in np.arange(n_lst_bins):
    lst_min = lst_start[i]
    lst_max = lst_end[i]
    lst_range = np.logical_and(lsts >= lst_min, lsts < lst_max)
    #print(lst_min*24./(2.*np.pi), lst_range.sum())
    
    if lst_range.sum() > min_time_samp:
        lst_min_str = Angle(lst_min, u.rad).to_string(unit=u.hr)
        lst_max_str = Angle(lst_max, u.rad).to_string(unit=u.hr)
        lst_filename = 'lst.'+JD+'.'+lst_min_str+'.'+lst_max_str+'.uvcRP.drift.uvh5'
        print(lst_filename)
        jd_to_select = jds[lst_range]
        files_to_read = path + np.unique(filenames[lst_range])
        uvd = UVData()
        uvd.read(files_to_read)
        lsts_in_file = np.unique(uvd.lst_array)
        uvd.select(time_range=[jd_to_select.min()-one_second, jd_to_select.max()+one_second])
        uvd.write_uvh5(os.path.join(path, lst_filename), clobber=True)
        # Gonna make this just drift for now
        #uvd.phase(lst_mid, lat, allow_rephase = True)

lst.2457549.10h36m00s.10h42m00s.uvcRP.drift.uvh5
lst.2457549.10h42m00s.10h48m00s.uvcRP.drift.uvh5
lst.2457549.10h48m00s.10h54m00s.uvcRP.drift.uvh5
lst.2457549.10h54m00s.11h00m00s.uvcRP.drift.uvh5
lst.2457549.11h00m00s.11h06m00s.uvcRP.drift.uvh5
lst.2457549.11h06m00s.11h12m00s.uvcRP.drift.uvh5
lst.2457549.11h12m00s.11h18m00s.uvcRP.drift.uvh5
lst.2457549.11h18m00s.11h24m00s.uvcRP.drift.uvh5
lst.2457549.11h24m00s.11h30m00s.uvcRP.drift.uvh5
lst.2457549.11h30m00s.11h36m00s.uvcRP.drift.uvh5
lst.2457549.11h36m00s.11h42m00s.uvcRP.drift.uvh5
lst.2457549.11h42m00s.11h48m00s.uvcRP.drift.uvh5
lst.2457549.11h48m00s.11h54m00s.uvcRP.drift.uvh5
lst.2457549.11h54m00s.12h00m00s.uvcRP.drift.uvh5
lst.2457549.12h00m00s.12h06m00s.uvcRP.drift.uvh5
lst.2457549.12h06m00s.12h12m00s.uvcRP.drift.uvh5
lst.2457549.12h12m00s.12h18m00s.uvcRP.drift.uvh5
lst.2457549.12h18m00s.12h24m00s.uvcRP.drift.uvh5
lst.2457549.12h24m00s.12h30m00s.uvcRP.drift.uvh5
lst.2457549.12h30m00s.12h36m00s.uvcRP.drift.uvh5
lst.2457549.12h36m00